In [1]:
import math
import random
import pandas as pd
import numpy as np 
#from sklearn.preprocessing import LabelEncoder,MinMaxScaler

#### Lendo os dados do arquivo para um dataframe
##### Obs: Editei o csv e inseri os nomes da colunas diretamente no arquivo por convenciência

In [2]:
#df = pd.read_csv('pima-indians-diabetes.csv',header=None)
df = pd.read_csv('carData.csv')

In [3]:
df

,buying,maint,doors,persons,lug_boot,safety,quality
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


In [4]:
attrs = df.columns[:-1].values
attrs

array(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'],
      dtype=object)

In [16]:
classes = pd.unique(df['quality'])
classes

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

#### Função para calcular a probabilidade multinomial

In [6]:
def calculateProbabilityTable(dataset,attr):
    
    
    #Valores unicos do atributo
    poss_vals = pd.unique(dataset[attr])
    
    classes = pd.unique(dataset['quality'])
 
    tabela_frequencia = {}

    for poss_val in poss_vals:
        tabela_frequencia[poss_val] = {}
        
        for classe in classes: 
            
            count_classe = len(dataset[dataset['quality'] == classe])
            
            sub_dataset = dataset.query(attr + "=='" +  poss_val + "' &" + "quality =='" + classe + "'")
            
            count_attr = len(sub_dataset)
            
            count = count_attr/count_classe
            
            tabela_frequencia[poss_val][classe] = count
            
        
    return tabela_frequencia

In [7]:
def calculateProbabilityTables(dataset):
    
    tables = {}
    
    for attr in df.columns[:-1]:
        
        tables[attr] = calculateProbabilityTable(dataset,attr)
    
    return tables

#### Com isso podemos calcular as probabilidades das classes

In [8]:
def calculateClassProbabilities(probabilityTables, inputVector, attrs,classes):
    
    probabilities = {}
    
    for classe in classes:
        
        classProb = 1.0
        
        for index,attr in enumerate(attrs):
            
            attrTable = probabilityTables[attr]
            
            print(attrTable[inputVector[index]])
            
            classAttrProb = attrTable[inputVector[index]][classe]
            
            classProb *= classAttrProb
            
        probabilities[classe] = classProb
        
    
    return probabilities

#### Agora finalmente podemos fazer a função de predição

In [9]:
def predict(probabilityTables, inputVector, attrs, classes):
    probabilities = calculateClassProbabilities(probabilityTables, inputVector,attrs,classes)
    bestLabel, bestProb = None, -1
    for classe in classes:
        classProb = probabilities[classe]
        if bestLabel is None or classProb > bestProb:
            bestProb = classProb
            bestLabel = classe
    return bestLabel

In [10]:
def getPredictions(probabilityTables, testSet, attrs,classes):
    predictions = []
    for i in range(len(testSet)):
        inputVector = testSet.iloc[i].values.tolist()
        #print(inputVector)
        result = predict(probabilityTables, inputVector,attrs,classes)
        predictions.append(result)
    return predictions

#### Função para dividir o dataset entre treino e teste

In [11]:
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = dataset.iloc[:trainSize-1]
    testSet = dataset.iloc[trainSize:]
    
    return [trainSet,testSet]

In [12]:
[train,test] = splitDataset(df,0.7)

#### Função para calcular a acurácia

In [13]:
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [14]:
probabilityTables = calculateProbabilityTables(train)

In [15]:
predictions = getPredictions(probabilityTables,test,attrs,classes)

{'unacc': 0.24336283185840707, 'acc': 0.37282229965156793, 'vgood': 1.0, 'good': 1.0}
{'unacc': 0.1747787610619469, 'acc': 0.2578397212543554, 'vgood': 0.07142857142857142, 'good': 1.0}
{'unacc': 0.27876106194690264, 'acc': 0.20557491289198607, 'vgood': 0.21428571428571427, 'good': 1.0}
{'unacc': 0.27654867256637167, 'acc': 0.4912891986062718, 'vgood': 0.5, 'good': 0.0}
{'unacc': 0.3241150442477876, 'acc': 0.3588850174216028, 'vgood': 0.35714285714285715, 'good': 0.3333333333333333}
{'unacc': 0.44579646017699115, 'acc': 0.0, 'vgood': 0.0, 'good': 0.0}
{'unacc': 0.24336283185840707, 'acc': 0.37282229965156793, 'vgood': 1.0, 'good': 1.0}
{'unacc': 0.1747787610619469, 'acc': 0.2578397212543554, 'vgood': 0.07142857142857142, 'good': 1.0}
{'unacc': 0.27876106194690264, 'acc': 0.20557491289198607, 'vgood': 0.21428571428571427, 'good': 1.0}
{'unacc': 0.27654867256637167, 'acc': 0.4912891986062718, 'vgood': 0.5, 'good': 0.0}
{'unacc': 0.3241150442477876, 'acc': 0.3588850174216028, 'vgood': 0.3

KeyError: 'low'